In [1]:
#import sympy as sp
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [3]:
theta, x, r, t = sp.symbols('theta x r t')

# Define the functions
I = (r*(sp.pi/2 + t + theta - sp.cos(t-theta)))/(1+sp.sin(t-theta))
f = r * sp.sin(t) - I*sp.cos(t)

# Define the equation
eq = sp.Eq(x, f)
display(eq)

# Real values
r_val = 14
x_val = 265/2
t_val = 3*sp.pi/2 - theta

# Solve the equation
display(eq.subs({r: r_val, x: x_val, t: t_val}))
sol = sp.solve(eq.subs({r: r_val, x: x_val, t: t_val}), theta)
display(sol)
(sp.re(sol[2])*180/sp.pi).evalf()

Eq(x, r*sin(t) - r*(t + theta - cos(t - theta) + pi/2)*cos(t)/(sin(t - theta) + 1))

Eq(132.5, -14*cos(theta) + 14*(sin(2*theta) + 2*pi)*sin(theta)/(1 - cos(2*theta)))

[-5.94482388814453 - 4.30133918527574e-23*I,
 -3.47995407262485 - 4.30133918527574e-23*I,
 0.338361419035052 - 4.30133918527574e-23*I,
 2.80323123455474 - 4.30133918527574e-23*I]

19.3866812607660

In [13]:
# --- Define constants ---
r_val = 14
x_val = 265 / 2

# --- Define the equations as functions ---
def I_func(t, theta):
    return (r_val*(np.pi/2 + t + theta - np.cos(t - theta))) / (1 + np.sin(t - theta))

def x_func(t, theta):
    return r_val * np.sin(t) - I_func(t, theta) * np.cos(t)

def y_func(t, theta):
    return -r_val * np.cos(t) - I_func(t, theta) * np.sin(t)

def equation(t, theta):
    return x_val - x_func(t, theta)

# --- Known values ---
theta_val = np.linspace(0.2, 0.334, 50)  # smaller sample here for demo
initial_guess = np.pi/2 + theta_val

# --- Solve for t numerically ---
solution_t = []
for i, th in enumerate(theta_val):
    sol = fsolve(equation, initial_guess[i], args=(th,))
    solution_t.append(sol[0])

solution_t = np.array(solution_t)

# --- Compute y_cut array ---
y_cut = y_func(solution_t, theta_val)
display(x_func(solution_t, theta_val))
# --- Define the CSV filename ---
csv_filename = "param_sweep_data.csv"

# --- Export to CSV in "2-column" COMSOL-friendly format ---
with open(csv_filename, "w") as f:
    # 1) First row: "theta_val" in col1, entire list in col2
    f.write("theta_val," + ",".join(str(val) for val in theta_val) + "\n")
    # 2) Second row: "y_func" in col1, entire list in col2
    f.write("y_func," + ",".join(str(val) for val in y_cut) + "\n")

print(f"CSV file '{csv_filename}' written successfully.")


array([132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5,
       132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5,
       132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5,
       132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5,
       132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5, 132.5,
       132.5, 132.5, 132.5, 132.5, 132.5])

CSV file 'param_sweep_data.csv' written successfully.


In [31]:
# Read the LVM file
raw_trunc = np.loadtxt('Truncated.txt', delimiter=',', skiprows=8)
raw_untrunc = np.loadtxt('Not truncated.txt', delimiter=',', skiprows=8)

# Extract the relevant data
SolNum = raw[:, 0]
AvgTemp = raw[:, 1]

SolNumUntrunc = raw_untrunc[:, 0]
AvgTempUntrunc = raw_untrunc[:, 1]

In [44]:
window = 10
N = AvgTemp.shape[0]
print(f'Number of data points: {N}')
frac = window / N  # Equivalent to MATLAB window parameter
print(f'Fraction: {frac}')

n_points = (0.50-0.34)/0.001
n_points = int(n_points)+2
print(f'Number of points: {n_points}')
theta_val_untrunc = np.linspace(0.34,0.50,n_points)
# Apply the LOWESS smoothing to the simulated temperature data
smoothed_temp = sm.nonparametric.lowess(AvgTemp, theta_val, frac=frac)
smoothed_temp_untrunc = sm.nonparametric.lowess(AvgTempUntrunc, theta_val_untrunc, frac=frac)

print(smoothed_temp[np.argmax(smoothed_temp[:, 1]), 0])
print(smoothed_temp_untrunc[np.argmax(smoothed_temp_untrunc[:, 1]), 0])

Number of data points: 50
Fraction: 0.2
Number of points: 161
Number of points: 161
0.2902448979591837
0.478


In [45]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=theta_val, y=AvgTemp, mode='lines', name='Temp'))
fig.add_trace(go.Scatter(x=smoothed_temp[:, 0], y=smoothed_temp[:, 1], mode='lines', name='Smoothed Temp'))
fig.add_trace(go.Scatter(x=theta_val_untrunc, y=AvgTempUntrunc, mode='lines', name='Temp Untrunc'))
fig.add_trace(go.Scatter(x=smoothed_temp_untrunc[:, 0], y=smoothed_temp_untrunc[:, 1], mode='lines', name='Smoothed Temp Untrunc'))
fig.show()

In [ ]:
theta_vals = np.linspace(0.1, 0.50, 2000)
x_theta = x_func(3*np.pi/2 -theta_vals, theta_vals)*2
display(x_theta)
index = np.argmax(x_theta <= 265)
print(index)
const_x = np.full(
  shape=np.shape(x_theta),
  fill_value=x_theta[index]
)

fig = go.Figure()
fig.add_trace(go.Scatter(x=theta_vals*180/np.pi, y=x_theta, mode='lines', name='Temp'))
fig.add_trace(go.Scatter(x=theta_vals*180/np.pi, y=const_x, mode='lines', name='Smoothed Temp'))

array([881.1137318 , 879.36001883, 877.61330802, ..., 183.61368131,
       183.54639149, 183.47915832], shape=(2000,))

1192
